In [1]:
import tensorflow as tf
print(tf.__version__)


2.19.0-dev20250121


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [3]:
import pandas as pd

df = pd.read_csv("loanprediction/Loan_default.csv")  # Replace with your actual filename
print(df.head())
print(df.info())


       LoanID  Age  Income  LoanAmount  CreditScore  MonthsEmployed  \
0  I38PQUQS96   56   85994       50587          520              80   
1  HPSK72WA7R   69   50432      124440          458              15   
2  C1OZ6DPJ8Y   46   84208      129188          451              26   
3  V2KKSFM3UN   32   31713       44799          743               0   
4  EY08JDHTZP   60   20437        9139          633               8   

   NumCreditLines  InterestRate  LoanTerm  DTIRatio    Education  \
0               4         15.23        36      0.44   Bachelor's   
1               1          4.81        60      0.68     Master's   
2               3         21.17        24      0.31     Master's   
3               3          7.07        24      0.23  High School   
4               4          6.51        48      0.73   Bachelor's   

  EmploymentType MaritalStatus HasMortgage HasDependents LoanPurpose  \
0      Full-time      Divorced         Yes           Yes       Other   
1      Full-time    

In [4]:
df = df[df['Default'].notna()]


In [5]:
print("Dataset shape:", df.shape)
print("Missing values per column:\n", df.isnull().sum())


Dataset shape: (255347, 18)
Missing values per column:
 LoanID            0
Age               0
Income            0
LoanAmount        0
CreditScore       0
MonthsEmployed    0
NumCreditLines    0
InterestRate      0
LoanTerm          0
DTIRatio          0
Education         0
EmploymentType    0
MaritalStatus     0
HasMortgage       0
HasDependents     0
LoanPurpose       0
HasCoSigner       0
Default           0
dtype: int64


In [10]:
df.dropna(inplace=True)


In [11]:
# Map Yes/No to 1/0
binary_map = {'Yes': 1, 'No': 0}
df['HasMortgage'] = df['HasMortgage'].map(binary_map)
df['HasDependents'] = df['HasDependents'].map(binary_map)
df['HasCoSigner'] = df['HasCoSigner'].map(binary_map)
df['Default'] = df['Default'].map(binary_map)

# Encode other categorical columns
categorical_cols = ['Education', 'EmploymentType', 'MaritalStatus', 'LoanPurpose']
le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

# Check final dataset shape again
print("Final dataset shape:", df.shape)

Final dataset shape: (0, 18)


In [14]:
import pandas as pd

df = pd.read_csv("loanprediction/Loan_default.csv")
print("Original shape:", df.shape)
print(df.head())


Original shape: (255347, 18)
       LoanID  Age  Income  LoanAmount  CreditScore  MonthsEmployed  \
0  I38PQUQS96   56   85994       50587          520              80   
1  HPSK72WA7R   69   50432      124440          458              15   
2  C1OZ6DPJ8Y   46   84208      129188          451              26   
3  V2KKSFM3UN   32   31713       44799          743               0   
4  EY08JDHTZP   60   20437        9139          633               8   

   NumCreditLines  InterestRate  LoanTerm  DTIRatio    Education  \
0               4         15.23        36      0.44   Bachelor's   
1               1          4.81        60      0.68     Master's   
2               3         21.17        24      0.31     Master's   
3               3          7.07        24      0.23  High School   
4               4          6.51        48      0.73   Bachelor's   

  EmploymentType MaritalStatus HasMortgage HasDependents LoanPurpose  \
0      Full-time      Divorced         Yes           Yes       

In [15]:
# Reload original CSV
df = pd.read_csv("loanprediction/Loan_default.csv")

# ✅ Step 1: Drop rows ONLY if 'Default' is missing
df = df[df['Default'].notna()]

# ✅ Step 2: Map 'Yes'/'No' to 1/0
yes_no_cols = ['HasMortgage', 'HasDependents', 'HasCoSigner', 'Default']
for col in yes_no_cols:
    if col in df.columns:
        df[col] = df[col].map({'Yes': 1, 'No': 0})

# ✅ Step 3: Label encode categorical features
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
categorical_cols = ['Education', 'EmploymentType', 'MaritalStatus', 'LoanPurpose']
for col in categorical_cols:
    if col in df.columns:
        df[col] = le.fit_transform(df[col].astype(str))

# ✅ Step 4: Fill remaining NaNs
df.fillna(method='ffill', inplace=True)

# ✅ Step 5: Split features and target
if 'LoanID' in df.columns:
    X = df.drop(['LoanID', 'Default'], axis=1)
else:
    X = df.drop('Default', axis=1)

y = df['Default']

print("Final shape of X:", X.shape)
print("Final shape of y:", y.shape)


Final shape of X: (255347, 16)
Final shape of y: (255347,)


C:\Users\Swapnil\AppData\Local\Temp\ipykernel_15196\3721458962.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [17]:
# Load fresh data
df = pd.read_csv("loanprediction/Loan_default.csv")

# ✅ Drop rows where 'Default' is NaN
df = df[df['Default'].notna()].copy()

# ✅ Encode 'Default' column if it's still Yes/No
if df['Default'].dtype == 'object':
    df['Default'] = df['Default'].map({'Yes': 1, 'No': 0})

# Check again
print("Any NaN in Default?:", df['Default'].isna().sum())
print("Value counts in Default:\n", df['Default'].value_counts())


Any NaN in Default?: 0
Value counts in Default:
 Default
0    225694
1     29653
Name: count, dtype: int64


In [18]:
# Encode other binary columns
binary_cols = ['HasMortgage', 'HasDependents', 'HasCoSigner']
for col in binary_cols:
    if df[col].dtype == 'object':
        df[col] = df[col].map({'Yes': 1, 'No': 0})

# Encode categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
categorical_cols = ['Education', 'EmploymentType', 'MaritalStatus', 'LoanPurpose']
for col in categorical_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# Fill any remaining NaN values
df.fillna(method='ffill', inplace=True)

# Prepare features and target
X = df.drop(columns=['LoanID', 'Default'], errors='ignore')
y = df['Default']

# Train-test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


C:\Users\Swapnil\AppData\Local\Temp\ipykernel_15196\4044691882.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_scaled, y_train)


LogisticRegression()

In [20]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predictions
y_pred = model.predict(X_test_scaled)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8858625416095555

Classification Report:
               precision    recall  f1-score   support

           0       0.89      1.00      0.94     45170
           1       0.62      0.03      0.06      5900

    accuracy                           0.89     51070
   macro avg       0.75      0.51      0.50     51070
weighted avg       0.86      0.89      0.84     51070


Confusion Matrix:
 [[45058   112]
 [ 5717   183]]


In [21]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predict and evaluate
rf_pred = rf_model.predict(X_test_scaled)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))


Random Forest Accuracy: 0.886822009007245


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build NN model
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
nn_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)


C:\Users\Swapnil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
5107/5107 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.8824 - loss: 0.3305 - val_accuracy: 0.8880 - val_loss: 0.3131
Epoch 2/20
5107/5107 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8838 - loss: 0.3178 - val_accuracy: 0.8878 - val_loss: 0.3112
Epoch 3/20
5107/5107 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8864 - loss: 0.3137 - val_accuracy: 0.8880 - val_loss: 0.3128
Epoch 4/20
5107/5107 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8861 - loss: 0.3135 - val_accuracy: 0.8878 - val_loss: 0.3120
Epoch 5/20
5107/5107 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8841 - loss: 0.3153 - val_accuracy: 0.8879 - val_loss: 0.3113
Epoch 6/20
5107/5107 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8853 - loss: 0.3133 - val_accuracy: 0.8881 - val_loss: 0.3118
Epoch 7/20
5107/5107 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8858 - loss: 0.3132 - val_accuracy: 0.8878 - val_loss: 0.3113
Epoch 8/20
5107/5107 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.8843 - loss: 0.3163 

In [23]:
# Evaluate on test data
loss, accuracy = nn_model.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {accuracy:.2f}")

# Predictions (rounded)
nn_pred = (nn_model.predict(X_test_scaled) > 0.5).astype("int32")


1596/1596 ━━━━━━━━━━━━━━━━━━━━ 2s 956us/step - accuracy: 0.8865 - loss: 0.3135 
Neural Network Accuracy: 0.89
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 1s 621us/step


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load your dataset
df = pd.read_csv("loanprediction/Loan_default.csv")  # Replace with your file name

# Drop LoanID (not useful for model training)
df.drop(columns=["LoanID"], inplace=True)

# Define column types
categorical_cols = ['Education', 'EmploymentType', 'MaritalStatus', 'LoanPurpose']
binary_cols = ['HasMortgage', 'HasDependents', 'HasCoSigner']
target_col = 'Default'

# Convert binary Yes/No to 1/0
for col in binary_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Split features and label
X = df.drop(columns=[target_col])
y = df[target_col]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train models
model = LogisticRegression()
model.fit(X_scaled, y)

rf_model = RandomForestClassifier()
rf_model.fit(X_scaled, y)

# ✅ Save all models and encoders
with open('logistic_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)


In [7]:

print(df["Education"].unique())


["Bachelor's" "Master's" 'High School' 'PhD']


In [9]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Define your input
education = "Bachelor"  # or "Bachelor's", or any other label you want to encode

# Step 2: List all possible education categories
all_education_labels = ["Bachelor's", "Master's", "High School", "PhD", "Bachelor"]

# Step 3: Fit the LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(all_education_labels)

# Step 4: Transform the input
education_encoded = label_encoder.transform([education])[0]

print("Encoded Education:", education_encoded)


Encoded Education: 0


In [25]:
nn_model.save('nn_model.h5')



In [ ]:
!streamlit run loanapp.py


In [4]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['EmploymentType', 'MaritalStatus', 'Education', 'LoanPurpose']  # Add others if needed

label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le


In [5]:
print("EmploymentType values seen during training:", label_encoders['EmploymentType'].classes_)


EmploymentType values seen during training: [0 1 2 3]
